In [1]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice

c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet\\exploitation'

In [3]:
data_f=pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f=pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

In [4]:
data_f.head()

,user_id,sig_id,fakeness
0,100,1,True
1,100,2,True
2,100,3,True
3,100,4,True
4,100,5,True


In [5]:
visual_f.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.000000,0.515184,0.0,0.446658,0.000000,1.533631,0.000000,0.830734,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.557626,2.271116,0.000000,0.000000,0.0
1,0.775465,0.491479,0.0,0.000000,0.000000,1.406079,0.000000,0.492579,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.852994,0.000000,0.114499,0.0
2,0.000000,0.000000,0.0,0.536446,0.000000,1.768807,0.000000,0.676527,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,2.392846,0.000000,0.226075,0.0
3,0.558339,0.000000,0.0,0.693711,0.166456,0.771811,0.643744,0.890198,1.242508,0.0,...,0.0,0.036022,0.000000,0.0,0.0,0.000000,1.391226,0.653054,0.000000,0.0
4,0.000000,0.000000,0.0,0.000000,0.000000,0.227453,1.312067,1.143076,1.002350,0.0,...,0.0,0.000000,0.067551,0.0,0.0,0.000000,0.892711,0.037287,0.196272,0.0


In [6]:
data_f_gen = data_f.loc[data_f['fakeness']==False]
data_f_fake = data_f.loc[data_f['fakeness']==True]
data_f_gen_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(1,13))]
data_f_gen_valid_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(13,25))]

In [13]:
scores_sum = 0
run_count = 0
for user_id in tqdm.tqdm(data_f['user_id'].unique(), ascii=True):
    y_train = data_f_gen_12['user_id']==user_id
    X_train = visual_f.loc[y_train.index]
    # clf = XGBClassifier(tree_method='gpu_hist',verbosity=2,predictor='gpu_predictor')
    clf = SVC(class_weight='balanced', gamma='scale')
    clf.fit(X_train, y_train)
    y_valid_fakes = data_f_fake.loc[data_f_fake['user_id']==user_id]
    y_valid = len(y_valid_fakes.index)*[False]
    X_valid = visual_f.loc[y_valid_fakes.index]
    score = clf.score(X_valid,y_valid)
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_gens = data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']==user_id]
    y_valid = len(y_valid_gens.index)*[True]
    X_valid = visual_f.loc[y_valid_gens.index]
    score = clf.score(X_valid,y_valid)
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_rand_fakes = data_f_gen_valid_12.loc[choice(data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']!=user_id].index,10)]
    y_valid = len(y_valid_rand_fakes.index)*[False]
    X_valid = visual_f.loc[y_valid_rand_fakes.index]
    score = clf.score(X_valid,y_valid)
    # print(score)
    scores_sum += score
    run_count += 1
    # print(scores_sum/run_count)
print(scores_sum/run_count)

100%|################################################################################| 300/300 [03:26<00:00,  1.60it/s]


0.9390376558024057


In [8]:
'''
y_valid_fakes = data_f_fake.loc[data_f_fake['user_id']==user_id]
y_valid = len(y_valid_fakes.index)*[False]
X_valid = visual_f.loc[y_valid_fakes.index]
'''

"\ny_valid_fakes = data_f_fake.loc[data_f_fake['user_id']==user_id]\ny_valid = len(y_valid_fakes.index)*[False]\nX_valid = visual_f.loc[y_valid_fakes.index]\n"

In [9]:
# X_train